In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
m_path = os.path.abspath(os.path.join("..\.."))
if m_path not in sys.path:
    sys.path.append(m_path)
    
# from models.rigging.weight_transfer import main
from models.rigging.weight_transfer_robust import transfer_weights

import open3d as o3d
import numpy as np
import torch

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from models.smplx_.body_models import SMPL

smpl = SMPL("models/smpl", gender="male")
src_weights = smpl.lbs_weights

In [3]:
import pickle
import torch

src_dict_path = "models/xcloth/no_git_test_data/88-1/88-1.pkl"

with open(src_dict_path, "rb") as f:
    src_dict = pickle.load(f)

pose = src_dict['pose']
trans = src_dict['trans']
scale = src_dict['scale']

with torch.no_grad():
    fin_pose= torch.FloatTensor(pose).unsqueeze(0)
    verts_T, T, pose_offsets = smpl(
        global_orient=fin_pose[:, :3], 
        body_pose=fin_pose[:, 3:]
    )

    ret_verts = verts_T.vertices
    ret_joints = verts_T.joints

    trans_verts = ret_verts.squeeze() #* scale + trans
    trans_joints = ret_joints.squeeze() #* scale + trans

smpl_mesh = o3d.geometry.TriangleMesh()
smpl_mesh.vertices = o3d.utility.Vector3dVector(trans_verts)
smpl_mesh.triangles = o3d.utility.Vector3iVector(smpl.faces.astype(int))
smpl_mesh.compute_vertex_normals()

joints = trans_joints[:24]
joints_pt = o3d.geometry.PointCloud()
joints_pt.points = o3d.utility.Vector3dVector(joints)
joints_pt.colors = o3d.utility.Vector3dVector(np.repeat([[0, 0, 1]], repeats=joints.shape[0], axis=0))

In [4]:
o3d.visualization.draw_geometries([smpl_mesh])

In [5]:
# glb_path = "C:/Users/User/Downloads/smpl_male_blend2.glb"
# glb = o3d.io.read_triangle_mesh(glb_path)

# o3d.visualization.draw_geometries([glb, o3d.geometry.LineSet.create_from_triangle_mesh(smpl_mesh)])

In [6]:
# source_mesh_path = "../no_git_test_data/1-1/smpl1.obj"
target_mesh_path = "models/xcloth/no_git_test_data/88-1/model_cleaned.obj"

# src_mesh = o3d.io.read_triangle_mesh(source_mesh_path)
tgt_mesh = o3d.io.read_triangle_mesh(target_mesh_path, True)
# tgt_mesh.compute_vertex_normals()
# align with smpl
tgt_mesh.vertices = o3d.utility.Vector3dVector(np.asarray(tgt_mesh.vertices) / scale - trans + verts_T.joints[:, 0].cpu().numpy())

o3d.visualization.draw_geometries([o3d.geometry.LineSet.create_from_triangle_mesh(smpl_mesh), joints_pt, tgt_mesh])

In [7]:
# tgt_weights, v_match, v_no_match = main(smpl_mesh, tgt_mesh, src_weights.cpu().numpy(), threshold_distance=0.5/scale)
try:
    tgt_weights, v_match, v_no_match = transfer_weights(smpl_mesh, tgt_mesh, src_weights.cpu().numpy(), return_match=True)
except:
    tgt_weights, v_match, v_no_match = transfer_weights(smpl_mesh, tgt_mesh, src_weights.cpu().numpy(), return_match=True, copy_all=True)

c:\Users\User\.conda\envs\fyp-xcloth\lib\site-packages\pytorch3d\ops\laplacian_matrices.py:128: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:607.)
  L = torch.sparse.FloatTensor(idx, cot.view(-1), (V, V))
c:\Users\User\CODE\FYP\models\rigging\weight_transfer_robust.py:137: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  Q = -L + L @ M_inv @ L


In [8]:
colors = np.zeros((tgt_weights.shape[0], 3))
colors[v_match] = np.array([0., .5, 0.])
colors[v_no_match] = np.array([.5, 0., 0])

tgt_mesh_seg = o3d.geometry.TriangleMesh(tgt_mesh)
tgt_mesh_seg.vertex_colors = o3d.utility.Vector3dVector(colors)
tgt_mesh_seg.triangle_uvs = o3d.utility.Vector2dVector()

o3d.visualization.draw_geometries([joints_pt, tgt_mesh_seg])

In [9]:
# from kaolin.metrics.trianglemesh import point_to_mesh_distance
# from trimesh.triangles import points_to_barycentric

# from typing import Tuple

# def comupte_transform(
#     target: torch.Tensor, 
#     vertices: torch.Tensor,
#     faces: torch.Tensor, 
#     T: torch.Tensor,
#     device: torch.device = torch.device("cuda")
# ) -> torch.Tensor:
#     """compute tranformation matrix for each vertex in the target mesh 
#     by finding the closest point on the reference faces.

#     Parameters
#     ----------
#     target : torch.Tensor
#         the vertices to compute the transformation for.
#         [n, 3]
#     vertices : torch.Tensor
#         the vertices in 3d position of the reference mesh
#         [m, 3]
#     faces : torch.Tensor
#         the faces in vertex indeice of the reference mesh
#         [f, 3]
#     T: torch,.Tensor
#         the transformation matices of each vertex
#         [m, 4, 4]

#     Returns
#     -------
#     torch.Tensor
#         the computed tranformation matrix of shape [n, 4, 4]
#     """

#     face_positions = vertices[faces]
#     _, face_ids, _ = point_to_mesh_distance(
#         target.to(device=device).unsqueeze(0),
#         face_positions.to(device=device).unsqueeze(0))
#     face_ids = face_ids.squeeze().to(device=target.device)
    
#     bary_coords = points_to_barycentric(face_positions[face_ids], target)
#     ref_T = T[faces[face_ids]]
#     target_T = (bary_coords * ref_T).sum(axis=1)
    
#     return target_T
    

In [10]:
# tgt_T = comupte_transform(
#     torch.as_tensor(np.asarray(tgt_mesh.vertices)),
#     torch.as_tensor(np.asarray(smpl_mesh.vertices)),
#     torch.as_tensor(np.asarray(smpl_mesh.triangles, dtype=int)),
#     T)

In [11]:
with torch.no_grad():
    fin_pose= torch.FloatTensor(pose).unsqueeze(0)
    verts_T, TT, pose_offsets_T = smpl()

    ret_verts = verts_T.vertices
    ret_joints = verts_T.joints

    trans_verts = ret_verts.squeeze() #* scale + trans
    trans_joints = ret_joints.squeeze() #* scale + trans

smpl_mesh_T = o3d.geometry.TriangleMesh()
smpl_mesh_T.vertices = o3d.utility.Vector3dVector(trans_verts)
smpl_mesh_T.triangles = o3d.utility.Vector3iVector(smpl.faces.astype(int))
smpl_mesh_T.compute_vertex_normals()

TriangleMesh with 6890 points and 13776 triangles.

In [12]:
def lbs(W, T, V, pose_offsets=None, inverse=False):
    # if inverse_rotation:
    #     view = T.view(-1, 4, 4)
    #     rotation = view[:, :3, :3]
    #     # translation = view[:, :3, -1]
    #     rotation = torch.linalg.inv(rotation)
    #     view[:, :3, :3] = rotation
    #     # T[:, :3, -1] = translation
    if inverse:
        T = torch.inverse(T.view(-1, 4, 4))
    T = (W @ T.reshape(-1, 16)).view(-1, 4, 4)
        
    if pose_offsets is not None:
        V = V + pose_offsets
    V_homo = torch.concat([V, torch.ones((V.shape[0], 1), device=V.device)], dim=-1).unsqueeze(dim=-1)
    V_homo = T @ V_homo
    return V_homo[:, :3, 0]

In [13]:
from models.xcloth.components.utils import create_o3d_pcd
from models.rigging.transform import interpolate_pose_offsets

device = "cuda"
T0 = T.cuda().squeeze()
T1 = TT.cuda().squeeze()

T_identity = torch.eye(4, device=device)
T_identity = T_identity.repeat(24, 1)

T2 = torch.clone(T1).view(-1, 4, 4)
T2[:, :3, -1] = torch.ones(24, 3)
T2 = T2.view(-1, 16)

# garment
W = torch.as_tensor(tgt_weights, device=device).to(dtype=torch.float32)
V = torch.as_tensor(np.asarray(tgt_mesh.vertices), device=device).to(dtype=torch.float32)
V_norm = lbs(W, T_identity, V)
pose_off2 = torch.as_tensor(interpolate_pose_offsets(smpl_mesh, tgt_mesh, pose_offsets.squeeze().numpy())).to(device=device)

V_T = lbs(W, T0, V_norm, pose_offsets=pose_off2)

# smpl
W_smpl = src_weights.cuda().squeeze()
V_smpl = torch.as_tensor(np.asarray(smpl_mesh.vertices), device=device).to(dtype=torch.float32)
v_smpl_norm = lbs(W_smpl, T_identity, V_smpl, inverse=True)

# results
pcd1 = create_o3d_pcd(V_norm.cpu().numpy() - pose_off2.cpu().numpy())
pcd1_g = create_o3d_pcd(V_T.cpu().numpy())
pcd2 = create_o3d_pcd((v_smpl_norm.cpu() - pose_offsets.squeeze()).numpy(), colors=np.ones(V_smpl.shape)/2)

new_mesh = o3d.geometry.TriangleMesh(tgt_mesh)
new_mesh.vertices = pcd1.points

new_smpl = o3d.geometry.TriangleMesh(smpl_mesh)
new_smpl.vertices = pcd2.points

o3d.visualization.draw_geometries([pcd1_g, pcd2])
o3d.visualization.draw_geometries([new_mesh, o3d.geometry.LineSet.create_from_triangle_mesh(new_smpl)])

In [14]:
joints_map = {
    "root": -1,
    "Pelvis": 0,
    "L_Hip": 1,
    "L_Knee": 4,
    "L_Ankle": 7,
    "L_Foot": 10,
    "R_Hip": 2,
    "R_Knee": 5,
    "R_Ankle": 8,
    "R_Foot": 11,
    "Spine1": 3,
    "Spine2": 6,
    "Spine3": 9,
    "Neck": 12,
    "Head": 15,
    "L_Collar": 13,
    "L_Shoulder": 16,
    "L_Elbow": 18,
    "L_Wrist": 20,
    "L_Hand": 22,
    "R_Collar": 14,
    "R_Shoulder": 17,
    "R_Elbow": 19,
    "R_Wrist": 21,
    "R_Hand": 23,
}

inverse_joints_map = { j: i for i, j in joints_map.items()}

In [15]:
import bpy
import bmesh

test_mesh_path = "models/data/test_data/assets/smpl_male_blend2.glb"

bpy.ops.wm.read_factory_settings(use_empty=True)
bpy.ops.import_scene.gltf(filepath=test_mesh_path)
# [print(o.name) for o in bpy.data.objects]

# bpy.data.objects["SMPL-mesh-male"].select_set(True)
obj = bpy.context.scene.objects["SMPL-mesh-male"]
bpy.ops.object.mode_set(mode="EDIT")

bm = bmesh.new()
bm.from_mesh(obj.data)

# clean original 
bmesh.ops.delete(bm, geom=bm.verts)

vertices = np.array(tgt_mesh.vertices)
vertices[:, -1] *= -1
vertices[:, [1, -1]] = vertices[:, [-1, 1]]
faces = np.asarray(tgt_mesh.triangles)

# add vertices and faces
bm_verts = []
bm_verts_id = []
for v in vertices:
    vert = bm.verts.new(v)
    vert.index = len(bm.verts)
    bm_verts_id.append(vert.index)
    bm_verts.append(vert)
    
for f in faces:
    bm.faces.new([
        bm_verts[f[0]],
        bm_verts[f[1]],
        bm_verts[f[2]]
    ])
    
bm.to_mesh(obj.data)  
bm.free()

# add sknning weights
for g in obj.vertex_groups:
    i = joints_map[g.name]
    if i == -1:
        continue
    w = tgt_weights[:, i]
    for i, v_id in enumerate(bm_verts_id):
        obj.vertex_groups[g.name].add([v_id], w[i], "REPLACE")

# add color / texture    
color_layer_name = "vertex_colors"
obj.data.vertex_colors.new(name=color_layer_name)
color_layer = obj.data.vertex_colors[color_layer_name]

inverse_id_map = { v: i for i , v in enumerate(bm_verts_id)}
for poly in obj.data.polygons:
    for vert_i_poly, vert_i_mesh in enumerate(poly.vertices):  
        if vert_i_mesh in inverse_id_map.keys():
            vert_i_loop = poly.loop_indices[vert_i_poly]
            color_layer.data[vert_i_loop].color = (*colors[inverse_id_map[vert_i_mesh]], 1.)
    
    
# reference from: https://stackoverflow.com/questions/67854896/how-do-i-set-the-base-colour-of-a-material-to-equal-vertex-colours-in-blender-2

material_name = "material0"
material = bpy.data.materials.new(name=material_name)
material.use_nodes = True
obj.data.materials.append(material)

# Get node tree from the material
nodes = material.node_tree.nodes
principled_bsdf_node = nodes.get("Principled BSDF")

# Get Vertex Color Node, create it if it does not exist in the current node tree
vertex_color_node = nodes.new(type="ShaderNodeVertexColor")

# Set the vertex_color layer we created at the beginning as input
vertex_color_node.layer_name = color_layer_name

# Link Vertex Color Node "Color" output to Principled BSDF Node "Base Color" input
links = material.node_tree.links
link = links.new(vertex_color_node.outputs[0], principled_bsdf_node.inputs[0])

obj.data.update()

bpy.ops.export_scene.gltf(filepath=f"{test_mesh_path.split('.')[0]}_edit.glb")

Data are loaded, start creating Blender stuff
glTF import finished in 0.32s
21:07:08 | ERROR: Draco mesh compression is not available because library could not be found at c:\Users\User\CODE\FYP\4.0\python\lib\site-packages\extern_draco.dll
21:07:09 | INFO: Starting glTF 2.0 export
21:07:09 | INFO: Extracting primitive: SMPL-shapes-male
21:07:28 | INFO: Primitives created: 1
21:07:28 | WARNING: There are more than 4 joint vertex influences.The 4 with highest weight will be used (and normalized).
21:07:31 | INFO: Finished glTF 2.0 export in 22.48082447052002 s



{'FINISHED'}

In [33]:
from models.xcloth.components.utils import o3d_to_skinned_glb

o3d_to_skinned_glb(tgt_mesh_seg, "models/data/test_data/assets/test.glb", armature_path=None)

21:31:03 | ERROR: Draco mesh compression is not available because library could not be found at c:\Users\User\CODE\FYP\4.0\python\lib\site-packages\extern_draco.dll
21:31:03 | INFO: Starting glTF 2.0 export
21:31:03 | INFO: Extracting primitive: NewMesh
21:31:03 | INFO: Primitives created: 1
21:31:03 | INFO: Finished glTF 2.0 export in 0.24127554893493652 s

